# Final Project

In [9]:
import mdptoolbox
import numpy as np
import scipy.constants as sc

## Unresolved Design Considerations

1. Would it make sense to use an even simpler radar state space model than the
   one considered in Selvi? We could use a 1D position/velocity space rather
   than 3D.

## Define the Radar Environment

The radar environment is defined by a set of possible postion states
$\mathcal{P}$ and a set of velocity states $\mathcal{V}$.

$\mathcal{P} = \{\mathbf{r_1}, \mathbf{r_2}, \dots, \mathbf{r_\rho}\}$

$\mathcal{V} = \{\mathbf{v_1}, \mathbf{v_2}, \dots, \mathbf{v_\nu}\}$

where $\rho$ is the number of possible position states and $\nu$ is the number
of possible velocities.

Each $\mathbf{r_i}, \mathbf{v_i}$ are 3-dimensional row vectors

$\mathbf{r_i} = \left[r_x, r_y, r_z \right]$

$\mathbf{v_i} = \left[v_x, v_y, v_z \right]$

where x, y and z represent the cross-range, down-range, and vertical dimensions,
respectively. 

In [10]:
class Radar:
  def __init__(self, pos, prf):
    self.position = np.array(pos)
    self.prf = prf
    self.max_range = sc.c/(2*prf)
  

In [12]:
# Radar system
radar = Radar(pos=np.array([0,0,0]),prf=2000)
# Number of position states
rho = 50
r = np.empty((rho,3))
r[:,1] = np.meshgrid
# Number of velocity states
nu = 10
# Discount factor
gamma = 0.9

# 